In [6]:
#!pip install semantic-kernel==0.3.10.dev

In [7]:
# reference link :
#https://towardsdatascience.com/a-pythonistas-intro-to-semantic-kernel-af5a1a39564d
#https://gist.github.com/Chris-hughes10/6dacd205f1da3cc3aec4fc45e57fb0b6
#https://learn.deeplearning.ai/microsoft-semantic-kernel/lesson/2/semantic-kernel-is-like-your-ai-cooking-kitchen
# The packages used were:

# dependencies:
#   - python=3.10.1.0
#   - pip:
#     - semantic-kernel==0.3.10.dev
#     - timm==0.9.5
#     - transformers==4.32.0
#     - sentence-transformers==2.2.2
#     - curated-transformers==1.1.0

<!-- One of the key concepts in Semantic Kernel is the kernel itself, which is the main object that we will use to orchestrate our LLM based workflows. Initially, the kernel has very limited functionality; all of its features are largely powered by external components that we will connect to. The kernel then acts as a processing engine that fulfils a request by invoking appropriate components to complete the given task. -->

One of the key concepts in Semantic Kernel is the kernel itself, which is the main object that we will use to orchestrate our LLM based workflows. Initially, the kernel has very limited functionality; all of its features are largely powered by external components that we will connect to. The kernel then acts as a processing engine that fulfils a request by invoking appropriate components to complete the given task.

In [1]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from IPython.display import display, Markdown

In [2]:
## initialize the kernel 
import semantic_kernel as sk
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

kernel = sk.Kernel()

In [3]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

In [4]:
## if we  want to avoid azure open ai semnatic kernel turn the below flag as false 
## since we are testing with openainand not azure openai , we are 
useAzureOpenAI = False
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenai", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

print("You made a kernel!")

You made a kernel!


The various services are :
1.text completion service: used to generate natural language
2.chat service: used to create a conversational experience
3.text embedding generation service: used to encode natural language into embeddings

Each type of service can support multiple models from different sources at the same time, making it possible to switch between different models, depending on the task and the preference of the user. If no specific service or model is specified, the kernel will default to the first service and model that was defined

In [5]:
##api_key,org_id
#We can see all of the currently registered services using the following methods:
def print_ai_services(kernel):
    print(f"Text completion services: {kernel.all_text_completion_services()}")
    print(f"Chat completion services: {kernel.all_chat_services()}")
    print(
        f"Text embedding generation services: {kernel.all_text_embedding_generation_services()}"
    )
print_ai_services(kernel)   ## as of now , no services  

Text completion services: ['openai']
Chat completion services: []
Text embedding generation services: []


In [6]:
# ## add the chat completion messages for hugging face 
## commented  for now , as of now  we will etst with openai only 
# import semantic_kernel as sk
# from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

# kernel = sk.Kernel()

# kernel.add_text_completion_service("huggingface", HuggingFaceTextCompletion("gpt2", task="text-generation"))

# print("You made an open source kernel using an open source AI model!")

## hence a kernel is just like  kitchen , which does not do anything  unless we start to cook with it 
## let us now proceed to cook with it 

In [6]:
#!pip install xformers

In [7]:
## we will consider the semantic functions for openai for this poc 
##Let's make a semantic function

sk_prompt = """
{{$input}}

Summarize the content above in less than 140 characters.
"""
summary_function = kernel.create_semantic_function(prompt_template = sk_prompt,
                                                    description="Summarizes the input to length of an old tweet.",
                                                    max_tokens=200,
                                                    temperature=0.0,
                                                    top_p=0.5)       
print("A semantic function for summarization has been registered.");

A semantic function for summarization has been registered.


In [8]:
sk_input = """
Let me illustrate an example. Many weekends, I drive a few minutes from my house to a local pizza store to buy 
a slice of Hawaiian pizza from the gentleman that owns this pizza store. And his pizza is great, but he always 
has a lot of cold pizzas sitting around, and every weekend some different flavor of pizza is out of stock. 
But when I watch him operate his store, I get excited, because by selling pizza, he is generating data. 
And this is data that he can take advantage of if he had access to AI.

AI systems are good at spotting patterns when given access to the right data, and perhaps an AI system could spot 
if Mediterranean pizzas sell really well on a Friday night, maybe it could suggest to him to make more of it on a 
Friday afternoon. Now you might say to me, "Hey, Andrew, this is a small pizza store. What's the big deal?" And I 
say, to the gentleman that owns this pizza store, something that could help him improve his revenues by a few 
thousand dollars a year, that will be a huge deal to him.
""";
# Text source: https://www.ted.com/talks/andrew_ng_how_ai_could_empower_any_business/transcript



In [9]:
##run_async method in Python that automatically calls a series of functions sequentially, all with the same context object.
summary_result = await kernel.run_async(summary_function, input_str=sk_input)

display(Markdown("###* " + str(summary_result)))

###* AI can help a small pizza store owner increase revenue by analyzing data and suggesting improvements based on customer patterns.

In [20]:
# ## had we used a hugging face completion  , then temperature would have been strictly postive > 0.0 , tat is 0.1
# # testing another text summarization 
# sk_input="Hannah Smith is a Dynamic data scientist, having hands on experience about complex Demand forecasting models , Image recognition , Regression , classification models as well as worked on text clustering , sentiment analysis and topic modelling.Worked on building demo recommendation engines , for proof of concept. Worked on telecom churn modelling . Have an explicit knowledge about restaurant business automation using machine learning.Worked in Cplex optimization.Have knowledge of deployment activities in Docker and Kubbernetes \
# Languages known are python , R , SQL, PLSQL,Unix.\
# Worked as a data engineer in tools like Hive , Mapreduce and spark.\
# Worked in proof-of-concepts related to retail client involving pyspark.\
# Worked on Shipping order management data processing and fetching the data through Kafka producer and mapping the same through Kafka consumer in spark streaming , displaying the grouped data through Socket IO server in a real time dashboard.\
# Have fair knowledge about Watson discovery , Watson assistant and Watson knowledge Studio.\
# Having good knowledge of visualization on Splunk , fetching data from splunk heavy forwarder as well as splunk db connect .\
# She is a technolgoy geek and is  enthusiastic  about quantum computing.She wants to know whetehr she will ever got to work in a quantum computing project?"

In [21]:
# #run_async method in Python that automatically calls a series of functions sequentially, all with the same context object.
# summary_result = await kernel.run_async(summary_function, input_str=sk_input)

In [10]:
## we can also call a proedure without caling the  kernel 
summary_result=summary_function(sk_input)


In [12]:
display(Markdown("### example 1  another way \n " + str(summary_result)))

### example 1  another way 
 AI can help a small pizza store owner increase revenue by analyzing data and suggesting improvements based on customer patterns.

# native functions: are not semantic functions , and they require lot of syntactic   trainings

In [13]:
from semantic_kernel.skill_definition import (
    sk_function,
    sk_function_context_parameter,
)

In [14]:
class ExoticLanguagePlugin:
    def word_to_pig_latin(self, word):
        vowels = "AEIOUaeiou"
        if word[0] in vowels:
            return word + "way"
        for idx, letter in enumerate(word):
            if letter in vowels:
                break
        else:
            return word + "ay"
        return word[idx:] + word[:idx] + "ay"
    @sk_function(
        description="Takes text and converts it to pig latin",
        name="pig_latin",
        input_description="The text to convert to pig latin",
    )
    def pig_latin(self, sentence:str) -> str:
        words = sentence.split()
        pig_latin_words = []
        for word in words:
            pig_latin_words.append(self.word_to_pig_latin(word))
        return ' '.join(pig_latin_words)

In [16]:
exotic_language_plugin = kernel.import_skill(ExoticLanguagePlugin(), skill_name="exotic_language_plugin")
pig_latin_function = exotic_language_plugin["pig_latin"]

print("creating a  pig_latin function , by calling exotic_language_plugin function or skill")


creating a  pig_latin function , by calling exotic_language_plugin function or skill


In [17]:
# lets us  once again find out the sk_input , summary_function
print(sk_input ,'\n',summary_function)


Let me illustrate an example. Many weekends, I drive a few minutes from my house to a local pizza store to buy 
a slice of Hawaiian pizza from the gentleman that owns this pizza store. And his pizza is great, but he always 
has a lot of cold pizzas sitting around, and every weekend some different flavor of pizza is out of stock. 
But when I watch him operate his store, I get excited, because by selling pizza, he is generating data. 
And this is data that he can take advantage of if he had access to AI.

AI systems are good at spotting patterns when given access to the right data, and perhaps an AI system could spot 
if Mediterranean pizzas sell really well on a Friday night, maybe it could suggest to him to make more of it on a 
Friday afternoon. Now you might say to me, "Hey, Andrew, this is a small pizza store. What's the big deal?" And I 
say, to the gentleman that owns this pizza store, something that could help him improve his revenues by a few 
thousand dollars a year, that will

In [18]:
final_result = await kernel.run_async(summary_function, pig_latin_function, input_str=sk_input) 
## here summary_function is a semantic function and piGg_latin_function is a native function

display(Markdown("### " + str(final_result)))

### AIway ancay elphay away allsmay izzapay orestay ownerway increaseway evenueray byay analyzingway ataday andway uggestingsay improvementsway asedbay onway ustomercay atterns.pay

In [20]:
str(summary_result)

'AI can help a small pizza store owner increase revenue by analyzing data and suggesting improvements based on customer patterns.'

##  now for swot ( strength , weakness , oppurtunity , threats ) we need to check a list of questions as below 
# Strengths
What unique recipes or ingredients does the pizza shop use?
What are the skills and experience of the staff?
Does the pizza shop have a strong reputation in the local area?
Are there any unique features of the shop or its location that attract customers?\n
# Weaknesses
What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)
Are there financial constraints that limit growth or improvements?
Are there any gaps in the product offering?
Are there customer complaints or negative reviews that need to be addressed?\n
# Opportunities
Is there potential for new products or services (e.g., catering, delivery)?
Are there under-served customer segments or market areas?
Can new technologies or systems enhance the business operations?
Are there partnerships or local events that can be leveraged for marketing?\n
# Threats
Who are the major competitors and what are they offering?
Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?
Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?
Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?

# a snapshot of swot answers
### 🔖 Opportunities and Threats

| **Opportunities**                                 | **Threats**                                                  |
| --- | ---|
| Untapped catering potential                       | Rising competition from cheaper pizza businesses nearby |
| Growing local tech startup community              | There's nearby street construction that will impact foot traffic |
| Unexplored online presence and order capabilities | Rising cost of cheese                                        |
| Upcoming annual food fair                         | No immediate local regulatory changes but it's election season |

In [23]:
## using LLM and semantic kernel , we can solve a lot of business problems 
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from IPython.display import display, Markdown

kernel = sk.Kernel()

useAzureOpenAI = False

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenai", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
print("Made another kernel!")

Made another kernel!


## jot down the SWOT questions

In [24]:
swot_interview= """
1. **Strengths**
    - What unique recipes or ingredients does the pizza shop use?
    - What are the skills and experience of the staff?
    - Does the pizza shop have a strong reputation in the local area?
    - Are there any unique features of the shop or its location that attract customers?
2. **Weaknesses**
    - What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)
    - Are there financial constraints that limit growth or improvements?
    - Are there any gaps in the product offering?
    - Are there customer complaints or negative reviews that need to be addressed?
3. **Opportunities**
    - Is there potential for new products or services (e.g., catering, delivery)?
    - Are there under-served customer segments or market areas?
    - Can new technologies or systems enhance the business operations?
    - Are there partnerships or local events that can be leveraged for marketing?
4. **Threats**
    - Who are the major competitors and what are they offering?
    - Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?
    - Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?
    - Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"""


## now make a semantic function that converts the analysis  to business  domain of {{$domain}}
## here domain is a different business domain 

In [26]:
sk_prompt = """
{{$input}}

Convert the analysis provided above to the business domain of {{$domain}}.
"""
shift_domain_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Translate an idea to another domain.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)
                   

In [27]:
my_context = kernel.create_new_context()

my_context['input'] = swot_interview
my_context['domain'] = "construction management"



In [28]:
my_context

SKContext(memory=<semantic_kernel.memory.null_memory.NullMemory object at 0x000001E40B4CA620>, variables=ContextVariables(variables={'input': '\n1. **Strengths**\n    - What unique recipes or ingredients does the pizza shop use?\n    - What are the skills and experience of the staff?\n    - Does the pizza shop have a strong reputation in the local area?\n    - Are there any unique features of the shop or its location that attract customers?\n2. **Weaknesses**\n    - What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)\n    - Are there financial constraints that limit growth or improvements?\n    - Are there any gaps in the product offering?\n    - Are there customer complaints or negative reviews that need to be addressed?\n3. **Opportunities**\n    - Is there potential for new products or services (e.g., catering, delivery)?\n    - Are there under-served customer segments or market areas?\n    - Can new technologies or systems enhance the bu

In [29]:
result = await kernel.run_async(shift_domain_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']}\n"+ str(result)))


### ✨ Shift the SWOT interview questions to the world of construction management
1. **Strengths**
    - What unique construction methods or materials does the company use?
    - What are the skills and experience of the construction team?
    - Does the company have a strong reputation in the local area?
    - Are there any unique features of the company or its projects that attract clients?
2. **Weaknesses**
    - What are the operational challenges of the company? (e.g., delays, budget overruns)
    - Are there financial constraints that limit growth or improvements?
    - Are there any gaps in the services offered?
    - Are there client complaints or negative reviews that need to be addressed?
3. **Opportunities**
    - Is there potential for new services or project types (e.g., green building, renovation)?
    - Are there under-served client segments or market areas?
    - Can new technologies or systems enhance the construction process?
    - Are there partnerships or local events that can be leveraged for marketing?
4. **Threats**
    - Who are the major competitors and what are they offering?
    - Are there potential negative impacts due to changes in the local area (e.g., zoning regulations, environmental concerns)?
    - Are there economic or industry trends that could impact the business negatively (e.g., increased material costs)?
    - Is there any risk due to changes in regulations or legislation (e.g., safety standards, building codes)?

In [30]:
sk_prompt = """
{{$input}}

Rewrite the text above to be understood by a {{$level}}.
"""


In [32]:
shift_reading_level_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Change the reading level of a given text.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)

my_context['input'] = swot_interview
my_context['domain'] = "construction management"
my_context["level"] = "child"

result = await kernel.run_async(shift_domain_function, shift_reading_level_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']} at the level of {my_context['level']}\n"+ str(result)))

### ✨ Shift the SWOT interview questions to the world of construction management at the level of child
1. Good things:
- Does the company build things in a special way or with special materials?
- Are the people who work there really good at building things?
- Do people in the area like the company and think they do a good job?
- Does the company make things that are different and interesting?
2. Bad things:
- Does the company sometimes have problems finishing things on time or spending too much money?
- Is it hard for the company to get enough money to grow or make things better?
- Are there things the company doesn't do that people want them to do?
- Do some people say bad things about the company or not like their work?
3. Good chances:
- Can the company start making new things or building in new ways (like making things that are good for the environment)?
- Are there people who want things built that the company doesn't usually make?
- Can the company use new tools or ways of building to make things better?
- Are there events or groups the company can work with to get more people to know about them?
4. Bad things that might happen:
- Are there other companies that are better or make things people like more?
- Are there rules or problems in the area that make it hard for the company to build things?
- Are there things that cost more money that the company needs to buy to make things?
- Are there new rules or laws that make it hard for the company to build things the way they usually do?

In [33]:
## next shifting the level to parent, for an fashion management  industry 
shift_reading_level_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Change the reading level of a given text.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)

my_context['input'] = swot_interview
my_context['domain'] = "fashion management"
my_context["level"] = "parent"

result = await kernel.run_async(shift_domain_function, shift_reading_level_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']} at the level of {my_context['level']}\n"+ str(result)))

### ✨ Shift the SWOT interview questions to the world of fashion management at the level of parent
1. What makes the fashion brand special? Do they use unique designs or materials? Does the team have good skills and experience? Do people like the brand?
2. Are there any problems with how the fashion brand works? Do they have trouble making things quickly or keeping staff? Do they have enough money? Are there any issues with the quality of their products or what customers say about them?
3. Are there any new things the fashion brand could do to grow? Could they make new things or work with other companies? Are there people who might like their products but don't know about them yet? Could they use new technology to help them work better? Are there any events or groups they could work with to get more people interested in their brand?
4. Are there any other companies that sell similar things? Are there any changes in what people like to wear that might hurt the fashion brand? Are there any changes in how much things cost that might make it harder for the fashion brand to make money? Are there any new rules or laws that might make it harder for the fashion brand to do business?

In [34]:
## next shifting the level to parent, for an education management  industry , from the perspective of a parent 
shift_reading_level_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Change the reading level of a given text.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)

my_context['input'] = swot_interview
my_context['domain'] = "education management"
my_context["level"] = "parent"

result = await kernel.run_async(shift_domain_function, shift_reading_level_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']} at the level of {my_context['level']}\n"+ str(result)))

### ✨ Shift the SWOT interview questions to the world of education management at the level of parent
As a parent, it's important to know what your child's school is doing well and where they could improve. Here are some things to consider:

1. Strengths: 
- Does the school have any special ways of teaching or learning that make it stand out?
- Are the teachers experienced and qualified?
- Does the school have a good reputation in the community?
- Are there any unique features of the school or location that make it appealing?

2. Weaknesses:
- Are there any challenges the school is facing, like not having enough resources or high staff turnover?
- Are there any financial issues that might limit the school's ability to grow or improve?
- Are there any areas where the school could improve its curriculum or extracurricular activities?
- Have there been any complaints or negative reviews from students or parents?

3. Opportunities:
- Are there any new programs or services the school could offer, like online learning or tutoring?
- Are there any groups of students or areas that the school could serve better?
- Could new technology or systems help improve the learning experience?
- Are there any partnerships or events in the community that the school could get involved with?

4. Threats:
- Are there other schools in the area that are competing with this school?
- Are there any changes happening in the community that could negatively impact the school, like a shift in demographics or other schools closing?
- Are there any economic or industry trends that could hurt the school, like less funding or higher costs?
- Are there any new regulations or laws that could affect the school, like changes to education policies or student safety rules?